In [1]:
#Capitulo 3:Introducción a los Algoritmos Cuánticos
#Algoritmo de Shor para la Factorización de Números
#Introducción al Algoritmo de Shor


In [2]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister 
from qiskit.circuit.library import QFT
from qiskit.primitives import Sampler
from math import gcd, log2, ceil
import numpy as np
from fractions import Fraction

def shor_circuit(N, a):
    n = ceil(log2(N))
    total_qubits = 2 * n
    qreg = QuantumRegister(total_qubits, 'q')
    creg = ClassicalRegister(n, 'c')
    circuit = QuantumCircuit(qreg, creg)
    
    # Aplicar puertas Hadamard a los primeros n qubits
    for i in range(n):
        circuit.h(qreg[i])
    
    # Exponenciación modular controlada
    controlled_modular_exponentiation(circuit, a, N, qreg, n)
    
    # Transformada de Fourier cuántica inversa
    circuit.append(QFT(n, inverse=True), qargs=qreg[:n])
    
    # Medir los primeros n qubits
    circuit.measure(qreg[:n], creg)
    
    return circuit

def controlled_modular_exponentiation(circuit, a, N, qreg, n):
    for i in range(n):
        exponent = pow(a, 2**i, N)
        modular_multiplication(circuit, exponent, N, qreg, i)

def modular_multiplication(circuit, exponent, N, qreg, control_qubit):
    target_qubit = (control_qubit + 1) % len(qreg)
    circuit.cp(exponent, control_qubit, target_qubit)

def find_period(N, a, measurements):
    if not isinstance(measurements, str):
        measurements = str(measurements)
    
    if not all(bit in '01' for bit in measurements):
        print(f"Error: Measurements '{measurements}' no es una cadena binaria válida.")
        return None
    
    decimal_value = int(measurements, 2) / (2 ** len(measurements))
    frac = Fraction(decimal_value).limit_denominator(N)
    r = frac.denominator
    
    if pow(a, r, N) == 1:
        return r
    else:
        return None

def shor_algorithm(N):
    if N % 2 == 0:
        return 2
    
    n = ceil(log2(N))
    
    for _ in range(100):
        a = np.random.randint(2, N)
        if gcd(a, N) != 1:
            return gcd(a, N)
        
        circuit = shor_circuit(N, a)
        sampler = Sampler()
        job = sampler.run(circuit, shots=100)
        result = job.result()
        counts = result.quasi_dists[0]
        measurements = max(counts, key=counts.get)
        
        r = find_period(N, a, measurements)
        if r:
            if r % 2 != 0:
                continue
            guess = gcd(pow(a, r//2, N) - 1, N)
            if 1 < guess < N:
                return guess
    
    return None

def factorize(N):
    print(f"\nIntentando factorizar: {N}")
    factor = shor_algorithm(N)
    if factor:
        print(f"Factor encontrado para {N}: {factor}")
        print(f"Los factores de {N} son {factor} y {N // factor}")
    else:
        print(f"No se pudo factorizar {N}")

if __name__ == "__main__":
    N = 15  # Número que queremos factorizar
    factorize(N)



Intentando factorizar: 15
Factor encontrado para 15: 5
Los factores de 15 son 5 y 3
